In [1]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import pandas as pd
import plotly.express as px
import copy
import base64
import io
import numpy as np
import itertools
from mlflow.sklearn import load_model


In [2]:
# On load les données
#data = pd.read_csv("concatenate_files.csv")

In [3]:
#data["ANNEES_AGE"] = (abs(data.DAYS_BIRTH) / 365.25)
#data["ANNEES_LAST_PHONE_CHANGE"] = round((abs(data.DAYS_LAST_PHONE_CHANGE) / 365.25), 2)

# On élimine les anciennes variables
#data.drop(["DAYS_BIRTH", "DAYS_LAST_PHONE_CHANGE"], axis = 1, inplace = True)

In [4]:
#infinity_indices = np.where(np.isinf(data))
#for row_ind, col_ind in zip(*infinity_indices):
#    data.iloc[row_ind, col_ind] = data.iloc[:,col_ind].median()

In [5]:
# On load le modèle 
#model = load_model("banking_model_20230901135647")

In [6]:
proba_threshold = 0.42

In [7]:
#data_dashboard = model[0].transform(data)

In [8]:
#data_dashboard["prediction"] = (model.predict_proba(data)[:, 1] >= (proba_threshold)).astype(int)
#data_dashboard["prediction_pret"] = data_dashboard["prediction"].apply(lambda x: "Pret" if x == 0 else "Non pret")
#data_dashboard["proba_pred_pret"] = model.predict_proba(data)[:, 0]
#data_dashboard["proba_pred_non_pret"] = model.predict_proba(data)[:, 1]

In [9]:
data_dashboard = pd.read_csv("dash_file.csv")

In [10]:
data_dashboard

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,FLAG_MOBIL,CNT_FAM_MEMBERS,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,...,POS_NAME_CONTRACT_STATUS_XNA_MEAN,INSTAL_PAYMENT_DIFF_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_COUNT,ANNEES_AGE,prediction,prediction_pret,proba_pred_pret,proba_pred_non_pret
0,100002,202500.0,24700.5,351000.0,0.018801,-3648.0,1,1.0,0,0,...,0.0,0.000,-49.0,-315.421053,19.0,25.902806,1,Non pret,0.276633,0.723367
1,100003,270000.0,35698.5,1129500.0,0.003541,-1186.0,1,2.0,0,0,...,0.0,0.000,-544.0,-1385.320000,25.0,45.900068,0,Pret,0.931821,0.068179
2,100004,67500.0,6750.0,135000.0,0.010032,-4260.0,1,1.0,0,0,...,0.0,0.000,-727.0,-761.666667,3.0,52.145106,0,Pret,0.710937,0.289063
3,100006,135000.0,29686.5,297000.0,0.008019,-9833.0,1,2.0,0,0,...,0.0,0.000,-12.0,-271.625000,16.0,52.032854,0,Pret,0.734408,0.265592
4,100007,121500.0,21865.5,513000.0,0.028663,-4311.0,1,1.0,0,0,...,0.0,22655.655,-14.0,-1032.242424,66.0,54.570842,0,Pret,0.627469,0.372531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100113,76500.0,6696.0,135000.0,0.028663,-9176.0,1,1.0,0,0,...,0.0,4944.690,-54.0,-1073.076923,13.0,41.442847,1,Non pret,0.449409,0.550591
96,100114,135000.0,22599.0,225000.0,0.030755,-7913.0,1,1.0,0,1,...,0.0,7881.300,-18.0,-114.200000,10.0,29.388090,1,Non pret,0.174195,0.825805
97,100115,90000.0,14683.5,315000.0,0.028663,-3071.0,1,2.0,0,0,...,0.0,0.000,-23.0,-130.000000,7.0,42.083504,1,Non pret,0.486823,0.513177
98,100116,243000.0,32017.5,900000.0,0.010006,-3390.0,1,2.0,0,0,...,0.0,29982.960,-57.0,-947.371681,113.0,40.386037,1,Non pret,0.474292,0.525708


In [11]:
data_dashboard.shape

(100, 104)

In [12]:
df = data_dashboard[:100]

In [13]:
# On calcul un score pour les clients ayant obtenus un prêt
df.loc[df.prediction_pret == "Pret", "score"] = round((df.loc[df.prediction_pret == "Pret", "proba_pred_pret"] - (1 - proba_threshold)) / (1 - proba_threshold), 2)

# On calcul un score pour les clients n'ayant pas obtenus un prêt
df.loc[df.prediction_pret == "Non pret", "score"] = round((((1 - proba_threshold) - df.loc[df.prediction_pret == "Non pret", "proba_pred_pret"]) / (1 - proba_threshold)) * -1, 2)

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_18827/72085085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df.prediction_pret == "Pret", "score"] = round((df.loc[df.prediction_pret == "Pret", "proba_pred_pret"] - (1 - proba_threshold)) / (1 - proba_threshold), 2)


In [14]:
df

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,FLAG_MOBIL,CNT_FAM_MEMBERS,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,...,INSTAL_PAYMENT_DIFF_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_COUNT,ANNEES_AGE,prediction,prediction_pret,proba_pred_pret,proba_pred_non_pret,score
0,100002,202500.0,24700.5,351000.0,0.018801,-3648.0,1,1.0,0,0,...,0.000,-49.0,-315.421053,19.0,25.902806,1,Non pret,0.276633,0.723367,-0.52
1,100003,270000.0,35698.5,1129500.0,0.003541,-1186.0,1,2.0,0,0,...,0.000,-544.0,-1385.320000,25.0,45.900068,0,Pret,0.931821,0.068179,0.61
2,100004,67500.0,6750.0,135000.0,0.010032,-4260.0,1,1.0,0,0,...,0.000,-727.0,-761.666667,3.0,52.145106,0,Pret,0.710937,0.289063,0.23
3,100006,135000.0,29686.5,297000.0,0.008019,-9833.0,1,2.0,0,0,...,0.000,-12.0,-271.625000,16.0,52.032854,0,Pret,0.734408,0.265592,0.27
4,100007,121500.0,21865.5,513000.0,0.028663,-4311.0,1,1.0,0,0,...,22655.655,-14.0,-1032.242424,66.0,54.570842,0,Pret,0.627469,0.372531,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100113,76500.0,6696.0,135000.0,0.028663,-9176.0,1,1.0,0,0,...,4944.690,-54.0,-1073.076923,13.0,41.442847,1,Non pret,0.449409,0.550591,-0.23
96,100114,135000.0,22599.0,225000.0,0.030755,-7913.0,1,1.0,0,1,...,7881.300,-18.0,-114.200000,10.0,29.388090,1,Non pret,0.174195,0.825805,-0.70
97,100115,90000.0,14683.5,315000.0,0.028663,-3071.0,1,2.0,0,0,...,0.000,-23.0,-130.000000,7.0,42.083504,1,Non pret,0.486823,0.513177,-0.16
98,100116,243000.0,32017.5,900000.0,0.010006,-3390.0,1,2.0,0,0,...,29982.960,-57.0,-947.371681,113.0,40.386037,1,Non pret,0.474292,0.525708,-0.18


On va donner des notes aux clients 

In [15]:
lettres = ['a', 'b', 'c', 'd', 'e', 'f']
signes = ['++', '+', '-', '--']

# On génére 2 dictionnaires
bon_clients = {}
mauvais_clients = {}

point_decr = (100 / ((len(lettres) * len(signes)) / 2)) / 100

point = 1
for l, s in itertools.product(lettres[:3], signes):
    key = l + s
    bon_clients[key] = round(point - point_decr, 2)
    point -= point_decr

print(bon_clients)

point = 0
for l, s in itertools.product(lettres[3:], signes):
    key = l + s
    mauvais_clients[key] = round(point, 2)*-1
    point += point_decr

print(mauvais_clients)

{'a++': 0.92, 'a+': 0.83, 'a-': 0.75, 'a--': 0.67, 'b++': 0.58, 'b+': 0.5, 'b-': 0.42, 'b--': 0.33, 'c++': 0.25, 'c+': 0.17, 'c-': 0.08, 'c--': -0.0}
{'d++': 0, 'd+': -0.08, 'd-': -0.17, 'd--': -0.25, 'e++': -0.33, 'e+': -0.42, 'e-': -0.5, 'e--': -0.58, 'f++': -0.67, 'f+': -0.75, 'f-': -0.83, 'f--': -0.92}


In [16]:
condition1 = df.prediction_pret == "Pret"
condition2 = df.prediction_pret == "Non pret"

for keys, items in bon_clients.items():
    df.loc[(condition1) & (df.score < items), "score_note"] = keys
    
for keys, items in mauvais_clients.items():
    df.loc[(condition2) & (df.score < items), "score_note"] = keys

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_18827/355609220.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[(condition1) & (df.score < items), "score_note"] = keys


In [17]:
df.loc[:, "score"] = round(df.loc[:, "score"]*100)

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_18827/2488000439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "score"] = round(df.loc[:, "score"]*100)


In [18]:
df[["score", "score_note"]]

,score,score_note
0,-52.0,e-
1,61.0,a--
2,23.0,c++
3,27.0,b--
4,8.0,c+
...,...,...
95,-23.0,d-
96,-70.0,f++
97,-16.0,d+
98,-18.0,d-


In [19]:
#df.to_csv("test.csv", index = False, sep = ",")

In [20]:
force = np.random.rand(99)

In [26]:
feature_importance  = [[v, round(r, 2)] for v, r in zip(var, force)]
feature_importance = pd.DataFrame(feature_importance, columns = ["Features", "Importance"]).sort_values("Importance", ascending = True)
feature_importance[:5]

,Features,Importance
11,EXT_SOURCE_2,0.01
39,BURO_CREDIT_TYPE_Loan for working capital repl...,0.01
66,PREV_NAME_TYPE_SUITE_Unaccompanied_MEAN,0.05
13,OBS_60_CNT_SOCIAL_CIRCLE,0.06
71,PREV_NAME_PRODUCT_TYPE_XNA_MEAN,0.07


In [27]:
df = None

In [55]:
file_df = None

In [29]:
feature_importance.Features.str[:20]

11            EXT_SOURCE_2
39    BURO_CREDIT_TYPE_Loa
66    PREV_NAME_TYPE_SUITE
13    OBS_60_CNT_SOCIAL_CI
71    PREV_NAME_PRODUCT_TY
              ...         
29    EMERGENCYSTATE_MODE_
41    ACTIVE_DAYS_CREDIT_M
47    PREV_AMT_ANNUITY_MAX
48    PREV_APP_CREDIT_PERC
35    BURO_DAYS_CREDIT_END
Name: Features, Length: 99, dtype: object

In [100]:
file_df.sort_values("score_note")

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,FLAG_MOBIL,CNT_FAM_MEMBERS,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,...,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_COUNT,ANNEES_AGE,prediction,prediction_pret,proba_pred_pret,proba_pred_non_pret,score,score_note
1,100003,270000.0,35698.5,1129500.0,0.003541,-1186.0,1,2.0,0,0,...,-544.0,-1385.320000,25.0,45.900068,0,Pret,0.931821,0.068179,61.0,a--
7,100010,360000.0,42075.0,1530000.0,0.003122,-4597.0,1,2.0,0,0,...,-774.0,-915.900000,10.0,51.608487,0,Pret,0.914679,0.085321,58.0,a--
99,100118,90000.0,26541.0,675000.0,0.026392,-217.0,1,1.0,0,0,...,-811.0,-1266.600000,20.0,61.522245,0,Pret,0.847919,0.152081,46.0,b+
76,100088,112500.0,6750.0,135000.0,0.022800,-4623.0,1,2.0,0,0,...,-34.0,-726.652174,23.0,53.297741,0,Pret,0.823976,0.176024,42.0,b+
6,100009,171000.0,41301.0,1395000.0,0.035792,-1213.0,1,3.0,0,0,...,-58.0,-864.411765,51.0,37.722108,0,Pret,0.823696,0.176304,42.0,b+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,100085,157500.0,28894.5,675000.0,0.016612,-3253.0,1,3.0,0,0,...,-1006.0,-1079.666667,6.0,30.628337,1,Non pret,0.251066,0.748934,-57.0,e-
0,100002,202500.0,24700.5,351000.0,0.018801,-3648.0,1,1.0,0,0,...,-49.0,-315.421053,19.0,25.902806,1,Non pret,0.276633,0.723367,-52.0,e-
15,100019,157500.0,20160.0,247500.0,0.020713,-3494.0,1,1.0,0,1,...,-716.0,-827.000000,8.0,23.895962,1,Non pret,0.235861,0.764139,-59.0,e--
61,100072,180000.0,44118.0,1080000.0,0.010006,-4557.0,1,1.0,0,0,...,-7.0,-464.533333,15.0,21.648186,1,Non pret,0.240505,0.759495,-59.0,e--


In [113]:
color_discrete_map = {"Pret": "blue", "Non pret": "red"}
nb_pret = file_df[file_df.prediction_pret == 'Pret'].shape[0]
nb_non_pret = file_df[file_df.prediction_pret == 'Non pret'].shape[0]

figure_score = px.histogram(file_df.sort_values("score_note"), x = "score_note",
                        color=file_df.sort_values("score_note")["prediction_pret"],  
                        color_discrete_map=color_discrete_map                       
                            # On ajoute un texte au dessus de la ligne              
                        ).add_annotation(text=f"Prêt (n={nb_pret})", x=3, y=15, showarrow=False,
                            font=dict(color="black", size=20)

                        # On ajoute un texte en dessous de la ligne
                        ).add_annotation(text=f"Non prêt (n={nb_non_pret})", x=13, y=15, showarrow=False,
                            font=dict(color="black", size=20))

figure_score.update_xaxes(title=dict(
            text = "Distribution des clients selont leur score note",
            font=dict(size=20, color="black"),
        ))
figure_score.update_yaxes(title=dict(
            text = "Comptage (nombre de client(s))",
            font=dict(size=20, color="black"),
        ))
figure_score

In [30]:
def feature_importance_threshold(df, feats, importance, variance_importance = 0.9):

    # On récupère le total
    variance_total = df[importance].sum()
    
    # Ce qui va nous permettre d'identifier les variables à afficher
    variance_threshold = variance_total * variance_importance
    
    # On trie
    df.sort_values(importance, ascending = False, inplace = True)
    
    # On mesure la somme continue de l'importance, en partant de la variable la plus importante
    cumsum = df[importance].cumsum()
    
    # On ne sélectionne que les features
    cols_num = [cols_num for cols_num, val in zip(cumsum.index, cumsum) if val <= variance_threshold]
    
    return df[df.index.isin(cols_num)]

def figure_feature_importance_dash(df, feats, importance, size, variance_importance = 0.9, color_point = "plotly", list_feat = None):
    ## 
    #    Fonction retournant un object de type figure
    #    On filtre les variables à afficher selon l'importance qu'ils ont sur le modèle (du plus important au moins)
    #    variance_importance = 0.5 : On affiche les variables qui ont 50 % du poids du modèle 
    #    variance_importance = 1 : On affiche l'ensemble des variables qui contribue au modèle
    ##
    
    
    # on ajuste la couleur des barres du graphique
    if color_point == "colorblind":
        color_bar = "#648FFF"
        color_bar_selected = "#FFB000"
    else :
        color_bar = "blue"
        color_bar_selected = "red"
    
    mask = feature_importance_threshold(df, feats, importance, variance_importance)
    
    # Permet de standardiser la taille du nom des variables, selon la taille du texte
    mask["truncated"] = mask[feats].str[:(60 - size)]
    
    figure_height = 200 + (mask.shape[0] * (size+1))
    
    # On instancie un objet px.bar avec les features filtrées
    figure = px.bar(mask.sort_values(importance, ascending = True), 
                    y=feats, 
                    x=importance,
                    height=figure_height, 
                   )
        
    # Si on a une liste de features sélectionnées, on les met en avant
    if list_feat:
        colors = [color_bar_selected if f in list_feat else color_bar for f in mask[feats]]
        figure.update_traces(marker_color=colors)
    else : 
        figure.update_traces(marker_color=color_bar)
        
    figure.update_yaxes(ticktext=mask["truncated"], tickvals=list(range(len(mask["truncated"]))))
    figure.update_layout(
        title=dict(
            text = (
                    f"{mask.shape[0]} variable{'(s)' if mask.shape[0] > 1 else ''} "
                    f"contribue{'nt' if mask.shape[0] > 1 else ''} à expliquer "
                    f"{variance_importance * 100} % du modèle"
                ),
            font=dict(size=8+size, color="black"),
            x=0.5,  # Center the title horizontally
            xanchor='center'
        ),
        font=dict(size=size, color="black"),
        yaxis=dict(
            tickmode='array',  
            tickvals=list(range(mask.shape[0])),  
            dtick=1,  
            automargin=True)
    
    )
    # On retourne la figure
    return figure


In [116]:
##### Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)


# App layout
app.layout = html.Div([
    
    # Titre dashboard
    html.Div(className='row', children="Utilisation du modèle pour prédire l'obtention d'un prêt",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
    html.Hr(), # Ligne horizontale
    
    # Upload liste de clients et leur caractéristiques
    html.Div(className='row',
        style={'textAlign': 'center', 'color': 'black', 'fontSize': 24}, children=[
            html.Div(className='row', children = "Chargez votre fichier clients ou le fichier test.csv"),
            # Boutton pour upload données 
            dcc.Upload(id="upload-data",children=[
                html.Button("Upload un fichier .csv", style={'color': 'black'})], multiple=False),
            html.Div(className='row', children="ou"),
            # Boutton pour upload les données tests
            html.Button("Upload test.csv", id="test_file_button", n_clicks=0, style={'color': 'black'}),
    ]),   

    html.Hr(), # Ligne horizontale
    
        # Choix des clients
    html.Div(className='row', children=[
        html.Div(className='client-selection', children=[
                html.Div("Choix d'un client :",
                         style={'textAlign': 'left', 'color': 'black', 'fontSize': 18}),
                dcc.Dropdown(
                        value=None,
                        style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                        placeholder="Sélection client(s)",
                        multi=True,
                        id='dropdown_client'
                ),
            ],
        ),
        # On affiche les 2 tableaux
        html.Div(id="table_client"),
        html.Div(className='row', children="Score de prédiction de l'obtention d'un prêt",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
        html.Div(id="table_prediction"),
        
            # Upload liste de clients et leur caractéristiques
        html.Div(className='row',
            style={'textAlign': 'center', 'color': 'black', 'fontSize': 24}, children=[
                html.Div(className='row', children = "Télécharger les fichiers de prédictions"),
                # Boutton pour download les résultats 
                html.Button("Result_prediction_all.csv", id="download-data_all-button", n_clicks = 0, style={'color': 'black'}),
                html.Button("Result_prediction_client.csv", id="download-data_client-button", n_clicks = 0, style={'color': 'black'}),
                dcc.Download(id="download-data"),
        ]),  
        
        html.Hr(), # Ligne horizontale
        # On affiche les différents graphs liés au résultat du modèle
        html.Div(className="figure_client", children=[
            # Option pour la taille des polices et des points
            html.Label("Taille police (axes et valeurs)"),
            dcc.Slider(id='font-size-slider', min=18, max=36, step=2, value=18,
                marks={i: str(i) for i in range(18, 37, 2)}),
            html.Label("Taille points (graphique)"),
            dcc.Slider(id='point-size-slider', min=4, max=16, step=2, value=8,
                marks={i: str(i) for i in range(4, 17, 2)}),
            # Option pour la palette de couleur
            html.Label("Sélectionnez une couleur de palette :"),
            dcc.RadioItems(id='color-palette-dropdown',
                style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                options=[
                    {'label': 'Défaut', 'value': 'plotly'},
                    {'label': 'Daltonien', 'value': 'colorblind'}],
                value='plotly', inline=True),
            html.Div(className='row', children="Représentation des clients :",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
            html.Label("Sélectionnez une métrique de score :"),
            dcc.RadioItems(id='variables_graph', 
                style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                options=[
                    {'label': 'Score', 'value': 'score'},
                    {'label': 'Proba_pred_pret', 'value': 'proba_pred_pret'},
                    {'label': 'Score_note', 'value': 'score_note'}],
                value='score', inline=True),
            dcc.Graph(id="graph_pred", style={'height': '600px', 'width': '100%', 'float': 'left'}),
            html.Label("Importances des variables pour le modèle (défaut = 0.9)"),
            dcc.Slider(id='feature-importance-slider', min=0, max=1, step=0.05, value=0.9,
                marks={i: str(round(i, 2)) for i in np.arange(0, 1.01, 0.05)}),
            dcc.Graph(id="graph_model", style={'width': '100%', 'float': 'left'}),
            #dcc.Graph(id="graph_client", style={'height': '600px', 'width': '50%', 'float': 'right'}),
            
        ]),
        # Choix des variables à représenter
        html.Div(className='variable-selection', 
            children=[
                html.Div("Afficher la distribution des variables définissant les clients",
                         style={'textAlign': 'center', 'color': 'black', 'fontSize': 18}),
                dcc.RadioItems(id='dropdown_fig_type', 
                    style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                    options=[
                        {'label': 'Strip', 'value': 'strip'},
                        {'label': 'Boxplot', 'value': 'boxplot'}],
                    value='strip', inline=True),
                dcc.Dropdown(
                        value=None,
                        style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                        placeholder="Sélection variable(s) (2 maximum)",
                        multi=True,
                        id='dropdown_variable'
                ),
                
                dcc.Graph(id="graph_variables", style={'height': '600px', 'width': '100%', 'float': 'left'}),
                
            ],
        ),
    ]),  
])


@app.callback(
    Output(component_id="table_client", component_property="children"),
    Output(component_id="table_prediction", component_property="children"),
    Output(component_id="graph_pred", component_property="figure"),
    Output(component_id="dropdown_variable", component_property="options"),
    Output(component_id="dropdown_client", component_property="options"),
    Output(component_id="dropdown_fig_type", component_property="options"),
    Output(component_id="dropdown_fig_type", component_property="value"),
    Output(component_id="graph_variables", component_property="figure"),
    Output(component_id="graph_model", component_property="figure"),
    Output(component_id="test_file_button", component_property="n_clicks"),
    Output(component_id="download-data", component_property="data"),
    Output(component_id="download-data_all-button", component_property="n_clicks"),
    Output(component_id="download-data_client-button", component_property="n_clicks"),
    
    Input(component_id="test_file_button", component_property="n_clicks"),
    Input(component_id="dropdown_client", component_property="value"),
    Input(component_id="variables_graph", component_property="value"),
    Input(component_id="dropdown_variable", component_property="value"),
    Input(component_id="upload-data", component_property="contents"),    
    Input(component_id="font-size-slider", component_property="value"), 
    Input(component_id="point-size-slider", component_property="value"),
    Input(component_id="color-palette-dropdown", component_property="value"),
    Input(component_id="dropdown_fig_type", component_property="value"),
    Input(component_id="feature-importance-slider", component_property="value"),
    Input(component_id="download-data_all-button", component_property="n_clicks"),
    Input(component_id="download-data_client-button", component_property="n_clicks"),
    
    
)



def update_table_and_button(n_clicks_file, selected_client, scoring_choisie, selected_variable, content, font_size, point_size, color_point, selected_affichage, variance_importance, n_clicks_dl_file_all, n_clicks_dl_file_clients):
    
    # On défini des couleurs selon la condition
    if color_point == "plotly" :
        color_discrete_map = {"Pret": "blue", "Non pret": "red"}
    
    #Développé par IBM, source : https://davidmathlogic.com/colorblind/)
    elif color_point == "colorblind" :
        color_discrete_map = {"Pret": '#648FFF', "Non pret": '#FFB000'}
    
    global file_df
    # On charge le fichier 
    if content is not None: 
        content_type, content_string = content.split(",")
        decoded = base64.b64decode(content_string)
        file_df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
     
    # On charge le fichier test.csv
    if n_clicks_file == 1:
        file_df = pd.read_csv("test.csv")
        n_clicks_file = 0
        
    # Par défaut, si aucun fichier de chargé
    if file_df is None :
        table1, table2 = dash_table.DataTable(), dash_table.DataTable()
        figure_score, figure_variables, figure_feat_imp = px.scatter(), px.scatter(), px.bar()
        option_drop_var, option_drop_clients, option_drop_var_graph = [], [], []
        selected_affichage = None
        return table1, table2, figure_score, option_drop_var, option_drop_clients, option_drop_var_graph, selected_affichage, figure_variables, figure_feat_imp, None, n_clicks_dl_file
    
    elif file_df is not None :
        # On met à jour la liste de dropdown
        # La liste de variable est dépendante de la liste de feature importance
        
        feat_imp_threshold = feature_importance_threshold(feature_importance, "Features", "Importance", variance_importance).Features.values
        option_drop_var=[{'label': str(var), 'value': var} for var in [col for col in feat_imp_threshold]]
        #option_drop_var=[{'label': str(var), 'value': var} for var in [col for col in file_df.iloc[:, 1:100].columns]]
        option_drop_clients=[{'label': str(client), 'value': client} for client in file_df['SK_ID_CURR']]
        cols = [0] + list(range(-6, 0))

        if selected_client == []:
            selected_client = None

        if (type(selected_client) != list) & (selected_client is not None):
            selected_client = [selected_client]

        if selected_client is None :
            table1 = dash_table.DataTable(
                data=file_df.iloc[:, :100].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
            table2 = dash_table.DataTable(
                data=file_df.iloc[:, cols].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
        else :
            filtered_df = file_df[file_df['SK_ID_CURR'].isin(selected_client)]
            table1 = dash_table.DataTable(
                data=filtered_df.iloc[:, :100].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
            table2 = dash_table.DataTable(
                data=filtered_df.iloc[:, cols].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
        if scoring_choisie == "score" :
            threshold = 0
            textH = threshold + 5
            textB = threshold - 5

        elif scoring_choisie == "proba_pred_pret":
            threshold = 1 - proba_threshold
            textH = threshold + 0.05
            textB = threshold - 0.05
            
        if scoring_choisie in ["score", "proba_pred_pret"]:
            figure_score = px.strip(file_df, y=scoring_choisie,
                        # On fait figurer le nom des clients et leur note
                        hover_data=["SK_ID_CURR", "score_note"],  
                        # On établit la coloration selon la condition de prêt 
                        color=file_df["prediction_pret"],  
                        color_discrete_map=color_discrete_map
                        # on actualise les valeurs de l'axe des x
                        ).update_xaxes(range=[-0.5, 0.5], tickvals=[0]  

                        # On ajoute une ligne horizontale
                        ).add_shape(type="line", x0=-1, x1=1, y0=threshold, y1=threshold,
                            line=dict(color="red", width=1)

                        # On ajoute un texte au dessus de la ligne
                        ).add_annotation(text="Threshold", x=-0.4, y=textH, showarrow=False,
                            font=dict(color="red", size=20)

                        # On ajoute un texte au dessus de la ligne              
                        ).add_annotation(text="Prêt", x=0.4, y=textH, showarrow=False,
                            font=dict(color="black", size=20)

                        # On ajoute un texte en dessous de la ligne
                        ).add_annotation(text="Non Prêt", x=0.4, y=textB, showarrow=False,
                            font=dict(color="black", size=20)

                        # On regroupe tout sur un seul tick
                        ).update_traces(offsetgroup=0.5)
            figure_score.update_traces(marker = dict(size=point_size))
            
        else : 
            nb_pret = file_df[file_df.prediction_pret == 'Pret'].shape[0]
            nb_non_pret = file_df[file_df.prediction_pret == 'Non pret'].shape[0]

            figure_score = px.histogram(file_df.sort_values("score_note"), x = "score_note",
                        color=file_df.sort_values("score_note")["prediction_pret"],  
                        color_discrete_map=color_discrete_map                       
                            # On ajoute un texte au dessus de la ligne              
                        ).add_annotation(text=f"Prêt (n={nb_pret})", x=3, y=15, showarrow=False,
                            font=dict(color="black", size=20)

                        # On ajoute un texte en dessous de la ligne
                        ).add_annotation(text=f"Non prêt (n={nb_non_pret})", x=13, y=15, showarrow=False,
                            font=dict(color="black", size=20))

            figure_score.update_xaxes(title=dict(
                        text = "Distribution des clients selont leur score note",
                        font=dict(size=20, color="black"),
                    ))
            figure_score.update_yaxes(title=dict(
                        text = "Comptage (nombre de client(s))",
                        font=dict(size=20, color="black"),
                    ))
        
        figure_score.update_layout(font=dict(size=font_size, color="black"))
        
        # On revient à la condition initiale
        selected_variable = None if selected_variable == [] else selected_variable

        # Si aucune variable n'est sélectionnée, nous n'affichons rien
        if selected_variable == None:
            figure_variables = px.scatter()
            option_drop_var_graph = []
            selected_affichage = None

        # Si une ou deux variables sont sélectionnées, on affiche soit un strip, soit un scatterplot
        elif (type(selected_variable) == list) :
            if len(selected_variable) == 1 :
                
                # On met à jour les options d'affichages pour une variable
                option_drop_var_graph=[
                        {'label': 'Strip', 'value': 'strip'},
                        {'label': 'Boxplot', 'value': 'boxplot'}]
                if(selected_affichage not in ["strip", "boxplot"]):
                    selected_affichage = "strip"
                
                # Type d'affichage
                if selected_affichage == "strip":
                    figure_variables = px.strip(file_df, y=selected_variable[0],
                        # On fait figurer le nom des clients et leur note
                        hover_data=["SK_ID_CURR", "score_note"],  
                        # On établit la coloration selon la condition de prêt 
                        color=file_df["prediction_pret"],  
                        color_discrete_map=color_discrete_map
                        # On regroupe tout sur un seul tick
                        ).update_traces(offsetgroup=0.5)
                    
                elif selected_affichage == "boxplot":
                    figure_variables = px.box(file_df, y=selected_variable[0],
                        # On fait figurer le nom des clients et leur note
                        hover_data=["SK_ID_CURR", "score_note"],  
                        # On établit la coloration selon la condition de prêt 
                        color=file_df["prediction_pret"],  
                        color_discrete_map=color_discrete_map
                        )
            else :
                
                # On met à jour les options d'affichages pour deux variables
                option_drop_var_graph=[
                        {'label': 'Scatterplot', 'value': 'scatter'},
                        {'label': 'Densité', 'value': 'density'}]
                if(selected_affichage not in ["scatter", "density"]):
                    selected_affichage = "scatter"
                
                # Type d'affichage
                if selected_affichage == "scatter":
                    figure_variables = px.scatter(file_df, x=selected_variable[0], y = selected_variable[1],
                            hover_data=["SK_ID_CURR", "score_note"],  
                            color=file_df["prediction_pret"], color_discrete_map=color_discrete_map)
                else :    
                    figure_variables = px.density_contour(file_df, x=selected_variable[0], y = selected_variable[1],
                        hover_data=["SK_ID_CURR", "score_note"],  
                        color=file_df["prediction_pret"], color_discrete_map=color_discrete_map)
                    
            figure_variables.update_layout(font=dict(size=font_size, color="black"))
            if selected_affichage != "density":
                figure_variables.update_traces(marker=dict(size=point_size))
            

        if (type(selected_client) == list) and (selected_client is not None) and (scoring_choisie != "score_note"):
            # On met à jour les graphiques avec les clients sélectionnés
            selected_data_p = file_df[(file_df["SK_ID_CURR"].isin(selected_client)) & (file_df["prediction_pret"] == "Pret")]
            selected_data_np = file_df[(file_df["SK_ID_CURR"].isin(selected_client)) & (file_df["prediction_pret"] == "Non pret")]

            # En créant une nouvelle trace
            if not selected_data_p.empty:
                # Mise à jour du graphique score avec les clients sélectionnés
                selected_trace_p = px.strip(selected_data_p, y=scoring_choisie,  
                                 color=selected_data_p["prediction_pret"], color_discrete_map=color_discrete_map,
                                 hover_data=["SK_ID_CURR", "score_note"],
                                 ).update_traces(marker_size=point_size+18, name = "Sélectionné(s)",
                                                marker_line_color="black", marker_line_width=2)
                figure_score.add_trace(selected_trace_p.data[0]).update_traces(offsetgroup=0.5)

                # Mise à jour du graphique variables avec les clients sélectionnés
                if type(selected_variable) == list :
                    if len(selected_variable) == 1 :
                        selected_trace_p = px.strip(selected_data_p, y=selected_variable[0],  
                                 color=selected_data_p["prediction_pret"], color_discrete_map=color_discrete_map,
                                 hover_data=["SK_ID_CURR", "score_note"],
                                 ).update_traces(marker_size=point_size+18, name = "Sélectionné(s)",
                                                marker_line_color="black", marker_line_width=2)
                        figure_variables.add_trace(selected_trace_p.data[0])
                        if selected_affichage == "strip":
                            figure_variables.update_traces(offsetgroup=0.5)

                    else :
                        selected_trace_p = px.scatter(selected_data_p, x=selected_variable[0], y = selected_variable[1],  
                                 color=selected_data_p["prediction_pret"], color_discrete_map=color_discrete_map,
                                 hover_data=["SK_ID_CURR", "score_note"],
                                 ).update_traces(marker_size=point_size+18, name = "Sélectionné(s)",
                                                marker_line_color="black", marker_line_width=2)
                        figure_variables.add_trace(selected_trace_p.data[0])


            if not selected_data_np.empty:
                # Mise à jour du graphique score avec les clients sélectionnés
                selected_trace_np = px.strip(selected_data_np, y=scoring_choisie, 
                                 color=selected_data_np["prediction_pret"], color_discrete_map=color_discrete_map,
                                 hover_data=["SK_ID_CURR", "score_note"],
                                 ).update_traces(marker_size=point_size+18, name = "Sélectionné(s)",
                                                marker_line_color="black", marker_line_width=2)
                figure_score.add_trace(selected_trace_np.data[0]).update_traces(offsetgroup=0.5)

                # Mise à jour du graphique variables avec les clients sélectionnés
                if type(selected_variable) == list :
                    if len(selected_variable) == 1 :
                        selected_trace_np = px.strip(selected_data_np, y=selected_variable[0],  
                                 color=selected_data_np["prediction_pret"], color_discrete_map=color_discrete_map,
                                 hover_data=["SK_ID_CURR", "score_note"],
                                 ).update_traces(marker_size=point_size+18, name = "Sélectionné(s)",
                                                marker_line_color="black", marker_line_width=2)
                        figure_variables.add_trace(selected_trace_np.data[0])
                        if selected_affichage == "strip":
                            figure_variables.update_traces(offsetgroup=0.5)

                    else :
                        selected_trace_np = px.scatter(selected_data_np, x=selected_variable[0], y = selected_variable[1],  
                                 color=selected_data_np["prediction_pret"], color_discrete_map=color_discrete_map,
                                 hover_data=["SK_ID_CURR", "score_note"],
                                 ).update_traces(marker_size=point_size+18, name = "Sélectionné(s)",
                                                marker_line_color="black", marker_line_width=2)
                        figure_variables.add_trace(selected_trace_np.data[0])
        
        # On prépare le graphique sur les features importances du modèle
        
        figure_feat_imp = figure_feature_importance_dash(feature_importance, "Features", "Importance", font_size, variance_importance, color_point, selected_variable)        
        
        list_var_dl = ["SK_ID_CURR", "score", "score_note"]
        
        # On télécharge l'ensemble des données clients
        if n_clicks_dl_file_all > 0:
            file_sent = dcc.send_data_frame(file_df[list_var_dl].to_csv, "Result_prediction_all.csv")
            n_clicks_dl_file_all = 0
        # On télécharge les données des clients sélectionnés
        elif n_clicks_dl_file_clients > 0 :
            print(selected_client)
            # Safegard pour éviter toute erreur de téléchargement
            if selected_client is not None:
                file_sent = dcc.send_data_frame(file_df.loc[file_df.SK_ID_CURR.isin(selected_client), list_var_dl].to_csv, "Result_prediction_client.csv")
            n_clicks_dl_file_clients = 0
        else :
            file_sent = None
            
        return table1, table2, figure_score, option_drop_var, option_drop_clients, option_drop_var_graph, selected_affichage, figure_variables, figure_feat_imp, n_clicks_file, file_sent, n_clicks_dl_file_all, n_clicks_dl_file_clients 

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_18827/756458583.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_18827/756458583.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2f/rfnwlqbn4kn5mss8s54qbyhh0000gn/T/ipykernel_18827/756458583.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
if file_df is not None :
    print("test")

In [ ]:
len(['AMT_INCOME_TOTAL'])

In [ ]:
# Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)


# App layout
app.layout = html.Div([
    
    # Titre dashboard
    html.Div(className='row', children="Utilisation du modèle pour prédire l'obtention d'un prêt",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
    
    # Premier tableau
    html.Div(className='row', children=[
        html.Div(
            className='client-selection',  # Add a class for styling
            children=[
                html.Div("Choix d'un client :",
                         style={'textAlign': 'left', 'color': 'black', 'fontSize': 18}),
                dcc.Dropdown(
                    options=[{'label': str(client), 'value': client} for client in df['SK_ID_CURR']],
                    value=None,
                    style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
                    placeholder="Sélectionne un client",
                    multi=True,
                    id='dropdown'
                ),
            ],
        ),
        html.Div(id="table_client")
    ]),
    
    # Boutton pour la prédiction
    html.Div(style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'height': '20vh'},
        children=[html.Div(style={'textAlign': 'center', 'color': 'black', 'fontSize': 18},
                children=[html.Button("Prédiction", id="prediction_button", n_clicks=0)],
                )]
            ),
    
    # Tableau à afficher
    html.Div(id="table_div"), 
    html.Div(id="radio_items_container"),
    html.Div(className='row', children=[
        html.Div(id="graph_pred", className="six columns"),
        html.Div(className="six columns", children=[
            html.Div(id="graph_feature_model"),
            html.Div(id="graph_feature_client"),
        ]),
    ], style={'display': 'flex', 'justifyContent': 'center'}),
    dcc.RadioItems(
        options=[],
        value='score',
        inline=True,
        style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
        id='variables_affichage_buttons'
    ),

])

# On crée une fonction callback afin de mettre à jour le boutton 
@app.callback(
    Output("table_client", "children"),
    Output("prediction_button", "children"),
    Output("prediction_button", "disabled"),
    Output("table_div", "children"),
    Output("graph_pred", "children"),
    Output("graph_feature_model", "children"),
    Output("graph_feature_client", "children"),
    Output("radio_items_container", "children"),
    #Output("variables_affichage_buttons", "children"),
    Input("dropdown", "value"),
    Input("prediction_button", "n_clicks"),
    #Input("variables_affichage_buttons", "children"),
    State("prediction_button", "children")
)

def update_table_and_button(selected_client, n_clicks, button_label):
    # Remise à 0
    var_choisie = "score"
    if selected_client == []:
        selected_client = None
    
    if (type(selected_client) != list) & (selected_client is not None) :
        selected_client = [selected_client]
        
    if selected_client is None :
        table1 = dash_table.DataTable(
            data=df.iloc[:, :100].to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    else :
        filtered_df = df[df['SK_ID_CURR'].isin(selected_client)]
        table1 = dash_table.DataTable(
            data=filtered_df.iloc[:, :100].to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    
    # Par défaut
    if n_clicks is None :
        n_clicks = 0
    if n_clicks == 0 :
        button_label = "Faire la prédiction"
        button_disabled = False
        table2 = None
        graph, graph2, graph3 = None, None, None
        radio_items = None
        radio_container_style = {'display': 'none'} 
        
    # Si la prédiction a été demandée
    else:
        button_label = "Prédiction réalisée"
        button_disabled = True
        

        cols = [0] + list(range(-6, 0))
        
        if var_choisie == "score":
            threshold = 0
        else: 
            threshold = 1 - proba_threshold
        
        if selected_client is None :
            table2 = dash_table.DataTable(
                data=df.iloc[:, cols].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
        else :
            filtered_df = df[df['SK_ID_CURR'].isin(selected_client)]
            table2 = dash_table.DataTable(
                data=filtered_df.iloc[:, cols].to_dict('records'),
                page_size=10,
                style_table={'overflowX': 'auto'}
            )
        
        # option des graphique des prédiction 
        
        radio_items = dcc.RadioItems(
            options=["score", "proba_pred_pret"],
            value='score',
            inline=True,
            style={'textAlign': 'left', 'color': 'black', 'fontSize': 15},
            id='radio_items_container'
            )
        radio_container_style = {'display': 'block'}
        
        # Graphique des prédiction
        graph = [dcc.Graph(
            figure=px.strip(
            df,
            y=var_choisie,
            hover_data=["SK_ID_CURR"],  # Pour afficher le nom des clients
            color=df["prediction_pret"],  # Condition for coloring
            color_discrete_map=color_discrete_map
            ).update_xaxes(
            range=[-0.5, 0.5],  # Set the x-axis range to [0, 1]
            tickvals=[0],  # Set the tick values
            ).add_shape(  # On ajoute une ligne horizontale
            type="line",
            x0=-1,
            x1=1,
            y0= threshold,
            y1= threshold,
            line=dict(
                color="red",
                width=1,
            ),
            ).add_annotation(  # Add text annotation below the line
            text="Threshold",
            x=-0.4,
            y= threshold + 0.05,  # Adjust the vertical position as needed
            showarrow=False,
            font=dict(
                color="red",
                size=20,
            ),
            ).add_annotation(  # Add text annotation above the line
            text="Prêt",
            x=0.4,
            y= threshold + 0.05,  # Adjust the vertical position as needed
            showarrow=False,
            font=dict(
                color="black",
                size=20,
            ),
        ).add_annotation(  # Add text annotation below the line
            text="Non Prêt",
            x=0.4,
            y= threshold - 0.05,  # Adjust the vertical position as needed
            showarrow=False,
            font=dict(
                color="black",
                size=20,
            ),
            ).update_traces(offsetgroup=0.5), # pour être sûr que ça groupe ensemble
            style={'height': '600px', 'width': '100%', 'float': 'left'} # Taille du graphique et position
                 
        )]
        graph2 = [dcc.Graph(
            figure=px.strip(),
            style={'height': '300px', 'width': '100%', 'float': 'right'}
        )]
        graph3 = [dcc.Graph(
            figure=px.strip(),
            style={'height': '300px', 'width': '100%', 'float': 'right'}
        )]
        
    return table1, button_label, button_disabled, table2, graph, graph2, graph3, radio_items

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
import base64
import io

# Import necessary libraries
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State

# Initialize the app with suppress_callback_exceptions=True
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

# Create a global variable to store the loaded data
file_df = None

# Define the app layout with tabs
app.layout = html.Div([
    # Title of the dashboard
    html.Div(className='row', children="Utilisation du modèle pour prédire l'obtention d'un prêt",
             style={'textAlign': 'center', 'color': 'black', 'fontSize': 30}),
    html.Hr(),  # Horizontal line

    # Tabs component with two tabs
    dcc.Tabs(id='tabs', value='tab-upload', children=[
        dcc.Tab(label='Upload Data', value='tab-upload'),
        dcc.Tab(label='Data Content', value='tab-content'),
    ]),

    # Container for tab content
    html.Div(id='tabs-content')
])

# Callback to switch between tabs and load the respective content
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    return tabs_content.get(tab, [])  # Return content for the selected tab

# Callback to parse and store the uploaded data
@app.callback(
    Output("data-content", "children"),
    Input("upload-data", "contents")
)
def parse_contents(content):
    global file_df
    if content is not None:
        content_type, content_string = content.split(",")
        decoded = base64.b64decode(content_string)
        file_df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        
        # Display a sample of the loaded data
        data_content = html.Div([
            html.H4("Sample Data Content:"),
            dcc.Graph(
                id='sample-data-table',
                config={'displayModeBar': False},
                figure={
                    'data': [
                        {
                            'x': file_df.columns,
                            'y': file_df.iloc[0],
                            'type': 'bar',
                            'name': 'Sample Data'
                        },
                    ],
                    'layout': {
                        'title': 'Sample Data Content',
                        'xaxis': {'title': 'Columns'},
                        'yaxis': {'title': 'Values'},
                    }
                }
            ),
        ])
        return data_content
    else:
        return html.Div("No data uploaded yet.")

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
